# Screening for side effects of Covid-19 drug candidates on cardiovascular development 

# Analysis notebook

In [1]:
%load_ext autoreload

In [2]:
%autoreload 3

In [3]:
%autoreload
import os
from glob import glob
import pandas as pd
import numpy as np

from glob import glob
from functions import analysis as ana
from functions.literature_search import searchPubmed
from functions.analysis import joined_find
import seaborn as sn
import matplotlib.pyplot as plt
import itertools
from importlib import reload
from functions.literature_search import find_intersection

import math
import datetime



C:\Users\aernst\Miniconda3\envs\TF25\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Pubmed search 

Add the list of drugs

In [4]:
drug_list = list(pd.read_excel("E:/Data_Drug_Screening_CovidBox/Master_Table/Druglist_LiteratureSearch.xlsx")["MODIFIED_NAME"])

In [ ]:
instance = searchPubmed("alexander.ernst@unibe.ch", drug_list, api_key="05b06f8adeace65e91dca5a0fac7d6ba330a")
df_literature = instance.multiple_context_search()[0]

In [ ]:
df_intersect = find_intersection(df_literature)

In [ ]:
# Uncomment if functions are not loaded
import sys
sys.path.append("Documents/GitHub/CoVasc_DrugScreen/functions/analysis.py")

# Get all the results from the Morphology assay
<b>Execute only if you have new data

Explanation:
    
One folder per experiment, the folders are identified by keywords which are referred to in the code to find the target content. 

Each folder has an "ExpID" at the end and contains 3 image folders, "Measurements"-folder and a "_PlateDescription.xlsx".

The images of one experiment are as tif files in 3 folders,     #1. 2x , #2. 4x and #3. 10x

Inside of the "Measurements"-folder are 4 tables "_HeartUnet_analysis","_ISVUnetAnalysis", "Summary_measurements", "Summary_heart" containing the analysis of the experiment.

The next cell loads all of them and accesses the "_10x" heartbeat movies to perform a frame rate correction on the heart beat counts. In this way alterations of the acquisition speed are corrected.

Afterwards the measurements are calibrated to "physical" units.

In [ ]:
# Uncomment to  save the table
#df_collection.to_excel("./tables/{}_Morphology_Assay_Collected_Measurements.xlsx".format(datetime.date.today()))

# Save the collected measurements of the morphology assay

In [19]:
df_collection = pd.read_excel("./tables/2022-07-28_Morphology_Assay_Collected_Measurements.xlsx")
df_collection 

,Drug,Well,Number ISV (Count),Edema,Body length (µm),Label,Heart Beats (Count),Frames (Count),Avg_Frametime (ms),Sum_Frametime (ms),...,image_name,ISV width (µm),ISV length (µm),Count_ISV,Experiment ID,TRIVIAL_NAME,Concentration (µM),Heart Rate (BPM),ISV area (µm^2),Repeat
0,Papaverine_1C02,A001,29.0,No,3969.975750,-A001--PO01--LO001--CO4--SL001--PX06500--PW020...,8.0,100,37.494949,3712.0,...,MAX_A001_1.tif,41.568419,118.746229,28,20,Papaverine,1.0,128.017241,4936.092954,20
1,Papaverine_1C02,A002,28.0,No,3773.302000,-A002--PO01--LO001--CO4--SL001--PX06500--PW020...,8.0,100,37.378788,3700.5,...,MAX_A002_1.tif,32.771138,166.579482,28,20,Papaverine,1.0,128.415079,5458.999150,20
2,Papaverine_1C02,A003,29.0,No,3772.054000,-A003--PO01--LO001--CO4--SL001--PX06500--PW020...,8.0,100,37.883838,3750.5,...,MAX_A003_1.tif,30.399782,163.101508,28,20,Papaverine,1.0,126.703106,4958.250297,20
3,Papaverine_1C02,A004,29.0,Yes,3812.562000,-A004--PO01--LO001--CO4--SL001--PX06500--PW020...,8.0,100,37.065657,3669.5,...,MAX_A004_1.tif,33.975478,173.823427,29,20,Papaverine,1.0,129.499932,5905.733928,20
4,Papaverine_1C02,A009,28.0,Yes,3764.117500,-A009--PO01--LO001--CO4--SL001--PX06500--PW020...,NaN,100,38.000000,3762.0,...,MAX_A009_1.tif,34.532891,178.950887,28,20,Papaverine,1.0,NaN,6179.691467,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,Control,E008,29.0,No,3624.194600,-E008--PO01--LO001--CO4--SL001--PX06500--PW020...,27.0,300,45.401338,13575.0,...,MAX_E008_1.tif,29.033603,100.793250,19,25,Control,1.0,118.939227,2926.391160,25
2424,Control,E009,29.0,No,3616.828150,-E009--PO01--LO001--CO4--SL001--PX06500--PW020...,31.0,300,37.954849,11348.5,...,MAX_E009_1.tif,34.190222,129.441127,19,25,Control,1.0,163.351985,4425.620921,25
2425,Control,E010,29.0,No,3539.585725,-E010--PO01--LO001--CO4--SL001--PX06500--PW020...,26.0,300,36.598662,10943.0,...,MAX_E010_1.tif,32.022479,143.883655,20,25,Control,1.0,142.081696,4607.511324,25
2426,Control,E011,29.0,No,3644.480450,-E011--PO01--LO001--CO4--SL001--PX06500--PW020...,28.0,300,37.061873,11081.5,...,MAX_E011_1.tif,28.229299,86.512676,17,25,Control,1.0,151.098678,2442.192172,25


In [6]:
collist = ["Drug","Experiment ID","Concentration (µM)","Repeat","Body length (µm)","Heart Rate (BPM)","Ejection fraction (%)","Number ISV (Count)","ISV area (µm^2)"]
df_standardized = ana.standardize_globalMedian(df_collection, collist, "Control", ["Concentration (µM)","Repeat"]).reset_index()
df_standardized = df_standardized[(df_standardized["Concentration (µM)"]==1.0) | (df_standardized["Drug"] == "Apilimod_1B03") | (df_standardized["Drug"] == "Niclosamide_1A02")]
df_standardized = df_standardized[~((df_standardized["Concentration (µM)"]==1.0) & (df_standardized["Drug"] == "Apilimod_1B03"))]
df_standardized

,Experiment ID,Drug,Concentration (µM),Repeat,Body length (µm),Ejection fraction (%),Heart Rate (BPM),ISV area (µm^2),Number ISV (Count)
0,20,Papaverine_1C02,1.0,20,3913.125125,24.807550,148.170909,4056.362334,29.0
1,20,Papaverine_1C02,1.0,20,3716.451375,14.421981,148.568747,4579.268530,28.0
2,20,Papaverine_1C02,1.0,20,3715.203375,20.284032,146.856774,4078.519677,29.0
3,20,Papaverine_1C02,1.0,20,3755.711375,28.766251,149.653599,5026.003308,29.0
4,20,Papaverine_1C02,1.0,20,3707.266875,19.955845,NaN,5299.960847,28.0
...,...,...,...,...,...,...,...,...,...
2423,25,Control,1.0,25,3658.141338,11.201110,128.043225,3549.614032,29.0
2424,25,Control,1.0,25,3650.774888,18.834188,172.455983,5048.843792,29.0
2425,25,Control,1.0,25,3573.532463,NaN,151.185694,5230.734196,29.0
2426,25,Control,1.0,25,3678.427188,21.499525,160.202676,3065.415044,29.0


In [7]:
listdrugs = df_standardized.Drug.unique()

measurement = ['Body length (µm)',
       'Heart Rate (BPM)', 'Ejection fraction (%)', 'Number ISV (Count)',"ISV area (µm^2)"]

df_mannwhitney = ana.calculate_stats(df_standardized, "mannWhitney", listdrugs, measurement)
df_effsize = ana.calculate_stats(df_standardized, "effSize" ,listdrugs, measurement)
df_deltamed = ana.calculate_stats(df_standardized, "deltaMedian", listdrugs, measurement)

df_multiscore = df_mannwhitney.applymap(math.log10).abs() * df_effsize.abs() * df_deltamed

df_sign = ana.sign_stars(df_mannwhitney)
df_sign_T = df_sign.T

In [8]:
listdrugs = ["Control","Remdesivir_1A06","Hydroxychloroquine_1F11","Lopinavir_1H04","Ritonavir_1A11","Favipiravir_1D10","Ivermectin_1F05","Ribavirin_1F08","Umifenovir_2E02","Baricitinib_2C09","Molnupiravir",
             "Apilimod_1B03","Astemizole_1D04","Manidipine_1G03","Cabozantinib_2G04","Regorafenib_1B04","Sorafenib_1F09","Ponatinib_2A07","Pimozide_2G07","Tacrolimus_2B07"]
cdict  = {i: "white" if i != "Control" else "red" for i in listdrugs}
df_covid = df_standardized.set_index("Drug").loc[listdrugs].reset_index()
df_sign_covid_T = df_sign.loc[listdrugs].T

In [9]:
measurement = ['Body length (µm)',
       'Heart Rate (BPM)', 'Ejection fraction (%)', 'Number ISV (Count)',"ISV area (µm^2)"]

df_covid_plot = df_covid.reset_index().pivot_table(index=["Drug","Experiment ID","Concentration (µM)","Repeat","index"],values=measurement).stack().reset_index()
df_covid_plot.columns = ["Drug","Experiment ID","Concentration (µM)","Repeat","ID","Measurement","Value"]
df_covid_plot

,Drug,Experiment ID,Concentration (µM),Repeat,ID,Measurement,Value
0,Apilimod_1B03,17,0.5,17,480,Body length (µm),3608.389688
1,Apilimod_1B03,17,0.5,17,480,Ejection fraction (%),12.840986
2,Apilimod_1B03,17,0.5,17,480,Heart Rate (BPM),26.142981
3,Apilimod_1B03,17,0.5,17,480,ISV area (µm^2),4252.060525
4,Apilimod_1B03,17,0.5,17,480,Number ISV (Count),28.000000
...,...,...,...,...,...,...,...
3062,Umifenovir_2E02,22,1.0,22,431,Body length (µm),3688.254700
3063,Umifenovir_2E02,22,1.0,22,431,Ejection fraction (%),17.976280
3064,Umifenovir_2E02,22,1.0,22,431,Heart Rate (BPM),149.293585
3065,Umifenovir_2E02,22,1.0,22,431,ISV area (µm^2),3885.684545


In [10]:
repeats = {}

for i in df_covid_plot["Drug"].unique():
    print(i)
    ids = df_covid_plot.reset_index().set_index("Drug").loc[i]["Experiment ID"].unique()
    for rep,x in  enumerate(ids):
        df_covid_plot = df_covid_plot.reset_index().set_index(["Experiment ID","Drug"])
        df_covid_plot["Repeat"].loc[(x,i)] = rep
df_covid_plot = df_covid_plot.reset_index()    

Apilimod_1B03
Astemizole_1D04
Baricitinib_2C09
Cabozantinib_2G04
Control
Favipiravir_1D10
Hydroxychloroquine_1F11
Ivermectin_1F05
Lopinavir_1H04
Manidipine_1G03
Molnupiravir


C:\Users\aernst\Miniconda3\envs\TF25\lib\site-packages\IPython\core\interactiveshell.py:2898: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
C:\Users\aernst\Miniconda3\envs\TF25\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Pimozide_2G07
Ponatinib_2A07
Regorafenib_1B04
Remdesivir_1A06
Ribavirin_1F08
Ritonavir_1A11
Sorafenib_1F09
Tacrolimus_2B07
Umifenovir_2E02


In [11]:
plt.figure(figsize=(20,40))
sns.set_theme(style="darkgrid")
sns.set_context("paper")
sns.set_theme(style="whitegrid")

g = sns.FacetGrid(df_covid_plot,row="Measurement",sharey=False,row_order=measurement,height=3,aspect=5,margin_titles=True,legend_out=True,despine=False)
g.map_dataframe(sns.stripplot, x="Drug",y="Value", hue="Repeat",palette=["black","darkgrey"],size=3,order=listdrugs)
g.map_dataframe(sns.boxplot, x="Drug",hue="Drug",y="Value",palette=cdict,fliersize=0,dodge=False,order=listdrugs,saturation=0.5)

g.refline(x=10.5,color='0')
for ax,rowname in zip(g.axes.flat,g.row_names):
    y_control = df_covid_plot[(df_covid_plot["Drug"]=="Control") & (df_covid_plot["Measurement"]==rowname)]["Value"]
    ax.hlines(xmin=0, y = y_control.median(), xmax=19.5,color="red")
    
    [ax.text(x = num, y = y_control.max(), s = stars, size = 'x-large', ha='center') for num,stars in enumerate(df_sign_covid_T.loc[rowname]) if stars != "ns"]


plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'  
);

NameError: name 'sns' is not defined

<Figure size 1440x2880 with 0 Axes>

# Get all the results from the Behavior assay

In [ ]:
dv_folder = "E:/Data_Drug_Screening_CovidBox/DanioVision"
directory = os.listdir(dv_folder)
df_program = pd.read_excel("E:/Data_Drug_Screening_CovidBox/DanioVision/20210115_A03_A11_ExpID0/150121_CoVasc_1min_bin_Bright_Dark_scheme.xlsx")
dict_program_melt = df_program.set_index("Bin").melt(var_name="Bin",value_name="Phase").set_index("Bin").to_dict()["Phase"]

for c, folders in enumerate(directory):
    #print(folders)
    if "_ExpID" in folders:
        
        expid = folders.split("_ExpID")[1]
        bin1_path = joined_find(dv_folder, folders, folder_measurements = "",keyword = "CoVasc_*1min*", file_type = ".xlsx",exclude=True)[0]
        filtered_path = list(filter(lambda x: "scheme" not in x ,bin1_path))
        map_path = joined_find(dv_folder, folders, folder_measurements = "",keyword = "_PlateMap", file_type = ".xlsx",exclude=True)[0]
        
        df_platemap = pd.read_excel(map_path)
        dict_platemap = df_platemap.dropna(axis=1).to_dict()
        df_bin1 = pd.read_excel(bin1_path, header = [0,1,2,3])
        
        df_bin1.columns = ['Trial', 'Well', 'Bin [1 sec]', 'Distance_moved [mm]', 'Velocity [mm/s]', 'Moving [s]','Not_Moving [s]']
        print(df_bin1.columns)
        df_bin1["Phase"] = df_bin1["Bin [1 sec]"].apply(ana.assign_phase, dict_program_clean=dict_program_melt)
        df_bin1["Drug"] = df_bin1["Well"].apply(ana.assign_compounds, dict_compound=dict_platemap)
        
        check = list(filter(lambda x: "_c" in x ,list(df_bin1["Drug"])))
        
        if check:
            df_bin1["Concentration (µM)"] = df_bin1["Drug"].str.split("_c", expand=True)[1].astype(float)
            df_bin1["Concentration (µM)"].loc[np.isnan(df_bin1["Concentration (µM)"])] = 1.0
        else:
            df_bin1["Concentration (µM)"] = 1.
        
        df_bin1["Drug"] = df_bin1["Drug"].str.split("__c",expand=True)[0]    
        df_bin1["Experiment ID"] = expid
        df_bin1 = df_bin1.loc[df_bin1["Phase"] != "Remove"]
        df_bin1 = df_bin1.loc[df_bin1["Drug"] != "Blank"]
        
        
        df_bin1_plot = df_bin1.groupby(["Experiment ID","Drug","Concentration (µM)","Bin [1 sec]"]).mean()
        df_bin1_phasemedian = df_bin1.groupby(["Experiment ID","Drug","Concentration (µM)","Well","Phase"]).median()
        
        
        if c==0:
            df_collect_lineplot = df_bin1_plot.copy()
            df_collect_phasemedian = df_bin1_phasemedian.copy()
        else:
            df_collect_lineplot = pd.concat([df_collect_lineplot, df_bin1_plot],axis=0)
            df_collect_phasemedian = pd.concat([df_collect_phasemedian, df_bin1_phasemedian],axis=0)
        print(map_path)
        print(df_collect_phasemedian.shape)
        #if expid == "17":
        #    break
        

# Filter out larvae that were not tracked properly

In [ ]:
threshold_toofast = 6
df_pivot = df_collect_phasemedian.reset_index().pivot(index=["Experiment ID", "Drug", "Concentration (µM)", "Well"],values=["Distance_moved [mm]", "Velocity [mm/s]", "Moving [s]" ,"Not_Moving [s]"], columns="Phase")
df_collect_phasemedian = df_collect_phasemedian.drop(list(df_pivot[(df_pivot["Velocity [mm/s]"]["Accomodation"]>threshold_toofast)|(df_pivot["Velocity [mm/s]"]["Dark"]>threshold_toofast)].index))

In [ ]:
df_collect_phasemedian = df_collect_phasemedian.reset_index()

In [ ]:
df_piv = df_collect_phasemedian.pivot(index=["Drug","Well","Experiment ID","Concentration (µM)"], columns="Phase", values=['Distance_moved [mm]', 'Velocity [mm/s]', 'Moving [s]',
       'Not_Moving [s]'])
    #df= df.reset_index()
df_piv.columns = df_piv.columns.map(('|'.join)).str.strip('|')

df_dv = df_piv.reset_index()
df_dv["Replicate"] = df_dv["Experiment ID"]
df_dv["Velocity|Ratio Light/Dark"] =  -np.log((df_dv["Velocity [mm/s]|Bright"]+0.001) / (df_standardized2["Velocity [mm/s]|Dark"]+0.001))

collist2 = list(df_dv.reset_index().columns)

df_standardized2 = ana.standardize_globalMedian(df_dv ,collist2, "Control", ["Concentration (µM)","Well","Replicate"]).reset_index()
df_standardized2 = df_standardized2[(df_standardized2["Concentration (µM)"]==1.0) | (df_standardized2["Drug"] == "Apilimod_1B03")| (df_standardized2["Drug"] == "Ivermectin_1F05")| (df_standardized2["Drug"] == "Niclosamide_1A02")]
df_standardized2 = df_standardized2[~((df_standardized2["Concentration (µM)"]==1.0) & (df_standardized2["Drug"] == "Apilimod_1B03"))]

In [ ]:

for i in df_standardized2["Drug"].unique():
 
    ids = df_standardized2.reset_index().set_index("Drug").loc[i]["Experiment ID"]
    
    
    if type(ids) == pd.Series:
        ids = ids.unique()

    for rep,x in  enumerate(ids):
        df_standardized2 = df_standardized2.reset_index().set_index(["Experiment ID","Drug"])
        df_standardized2["Replicate"].loc[(x,i)] = rep
df_standardized2 = df_standardized2.reset_index()    

In [ ]:
alldrugs = df_standardized2.Drug.unique()

measurement2 = ["Moving [s]|Accomodation","Velocity [mm/s]|Bright", "Velocity [mm/s]|Dark","Velocity|Ratio Light/Dark"]

df_mannwhitney2 = ana.calculate_stats(df_standardized2, "mannWhitney", alldrugs, measurement2)
df_effsize2 = ana.calculate_stats(df_standardized2, "effSize" ,alldrugs, measurement2)
df_deltamed2 = ana.calculate_stats(df_standardized2, "deltaMedian", alldrugs, measurement2)

df_multiscore = df_mannwhitney2.applymap(math.log10).abs() * df_effsize2.abs() * df_deltamed2

df_sign2 = ana.sign_stars(df_mannwhitney2)
df_sign_T2 = df_sign2.T

In [ ]:
listdrugs = ["Control","Remdesivir_1A06","Hydroxychloroquine_1F11","Lopinavir_1H04","Ritonavir_1A11","Favipiravir_1D10","Ivermectin_1F05","Ribavirin_1F08","Umifenovir_2E02","Baricitinib_2C09","Molnupiravir",
             "Apilimod_1B03","Astemizole_1D04","Manidipine_1G03","Cabozantinib_2G04","Regorafenib_1B04","Sorafenib_1F09","Ponatinib_2A07","Pimozide_2G07","Tacrolimus_2B07"]
cdict2  = {i: "white" if i != "Control" else "red" for i in listdrugs}
df_covid2 = df_standardized2.set_index("Drug").loc[listdrugs].reset_index()
df_sign_covid_T2 = df_sign2.loc[listdrugs].T

In [ ]:
select = df_collect_phasemedian.loc["04"]
sns.swarmplot(data=select.reset_index(),x="Drug", y = "Velocity [mm/s]", hue ="Phase")

plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'  
);

In [ ]:
df_collect.to_csv("E:/Data_Drug_Screening_CovidBox/Master_Table/CovidDrugScreen_Results_MeanActivity_Daniovision.csv")

In [ ]:
df3 = df_collect.copy()

In [ ]:
x = ["Remdesivir_1A06","Bemcentinib_1A04"]

experiment_ids3 = np.sort(df3.loc[x].reset_index()["Experiment ID"].unique())


listexperiments1 = {expid: df3.reset_index().set_index("Experiment ID").loc[expid1]["Drug"].unique() for expid1 in experiment_ids3 }

In [ ]:
list(set(list(listexperiments['0'])).intersection(['Bemcentinib_1A04', 'Ciclesonide_1A08']))

In [ ]:
pd.read_csv("E:/Data_Drug_Screening_CovidBox/Master_Table/CovidDrugScreen_Results_MeanActivity_Daniovision.csv").set_index(["Experiment ID","Drug"])

In [ ]:
plt.figure(figsize=(30,10))
sns.lineplot(data = df_bin1_plot, x="Bin [1 sec]", y="Velocity [mm/s]", hue="Drug")

plt.xticks(
    rotation = 90, 
    horizontalalignment = 'right',
    fontweight = 'light',
    fontsize = 10  
);

plt.yticks(
    fontsize = 10  
);


In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
fig = px.line(df_collect.loc["0"].reset_index(), x="Bin [1 sec]", y="Velocity [mm/s]", color='Drug')
fig.show()
